# FPA Monthly Registered Biofertilizer Report

This report allows visibility over current registered azospirillum-based products and their product descriptions in the Philippines. It is sent via email automatically on a monthly basis.

## Main Elements

1. Data Source: https://fpa.da.gov.ph/resources/reports/registered-products/
2. Google App Script - Scheduled monthly script that connects to the data source, extracts the data, loads into a google sheet, and sends the contents as an email to certain recepients
3. Google Sheet (https://docs.google.com/spreadsheets/d/1NqiBb-F2IMIOAj8JbPxfKXF0mF4sxVinqdVSo009dcg/edit?usp=sharing)
4. Automated Email


## Process Map

### 1. Data extraction from the FPA website

In [ ]:
// 1. Fetch the HTML of the FPA product listing page
  const url = 'https://fpa.da.gov.ph/resources/reports/registered-products/';
  const html = UrlFetchApp.fetch(url).getContentText();
  Logger.log("Fetched page HTML length: " + html.length);
  
  // 2. Find the latest XLSX link
  const match = html.match(/https:\/\/fpa\.da\.gov\.ph\/wp-content\/uploads\/[0-9]{4}\/[0-9]{2}\/FOR-POSTING_FERTILIZER_PRODUCT_LISTING_AS_OF_[^"']+\.xlsx/);
  if (!match) throw new Error("No XLSX link found on page");
  const xlsxUrl = match[0];
  Logger.log("Found XLSX URL: " + xlsxUrl);
  
  // 3. Download the XLSX file
  const blob = UrlFetchApp.fetch(xlsxUrl).getBlob();
  const tempFile = DriveApp.createFile(blob).setName("temp_fertilizer.xlsx");
  Logger.log("Temp XLSX saved to Drive: " + tempFile.getId());

### 2. Load extracted data into the Google Sheet

In [ ]:
 // 4. Convert to a temporary Google Sheet
  const convertedFile = Drive.Files.copy(
  { title: tempFile.getName(), mimeType: MimeType.GOOGLE_SHEETS },
  tempFile.getId(),
  { convert: true }
  );
  Logger.log("Converted file ID: " + convertedFile.id);
  Logger.log("Converted MIME type: " + convertedFile.getMimeType());

  // 5. Wait until the sheet is ready
  const sheetFile = waitForSheet(convertedFile.id, 10); // retry up to 10 times
  const sourceSheet = sheetFile.getSheets()[0];
  sourceSheet.deleteRows(1, 6);
  
  // 6. Read data
  const data = sourceSheet.getDataRange().getValues();
  
  // 7. Find column index for Guaranteed Analysis
  const header = data[0].map(h => h.toString().trim().toLowerCase());
  const guaranteedIndex = header.indexOf("guaranteed analysis");
  if (guaranteedIndex === -1) throw new Error("Guaranteed Analysis column not found");
  
  // 8. Filter rows that contain Azospirillum
  const filtered = data.filter((row, idx) => {
    if (idx === 0) return true; // keep header
    return row[guaranteedIndex].toString().toLowerCase().includes("azospirillum");
  });
  
  // 9. Paste to target sheet
  const target = SpreadsheetApp.openById(TARGET_SHEET_ID).getSheets()[0];
  target.clearContents();
  target.getRange(1, 1, filtered.length, filtered[0].length).setValues(filtered);
  
  // 10. Clean up temp file
  DriveApp.getFileById(tempFile.getId()).setTrashed(true);
  DriveApp.getFileById(convertedFile.id).setTrashed(true);
  
  Logger.log("Imported " + (filtered.length - 1) + " Azospirillum products");

  Logger.log("Filtered Azospirillum data imported successfully.");


### 3. Send contents via email as both HTML and attached Excel File

In [ ]:
  const TARGET_SHEET_ID = '1NqiBb-F2IMIOAj8JbPxfKXF0mF4sxVinqdVSo009dcg';
  const sheetName = 'Sheet1'; // Sheet name
  const recipients = ['xxx@gmail.com','yyy@gmail.com','zzz@gmail.com']; // recepients

  const ss = SpreadsheetApp.openById(TARGET_SHEET_ID);
  const sheet = ss.getSheetByName(sheetName);
  const data = sheet.getDataRange().getValues();

  //11. Convert to HTML table for email body
  let htmlTable = '<table border="1" cellpadding="5" cellspacing="0" style="border-collapse:collapse;">';
  data.forEach(row => {
    htmlTable += '<tr>';
    row.forEach(cell => {
      htmlTable += `<td>${cell || ''}</td>`;
    });
    htmlTable += '</tr>';
  });
  htmlTable += '</table>';

  //12. Create temporary Excel file
  const tempSs = SpreadsheetApp.create("Temp Azospirillum Data");
  tempSs.getSheets()[0].getRange(1, 1, data.length, data[0].length).setValues(data);
  const tempFileId = tempSs.getId();
  Utilities.sleep(2000); // wait 2 seconds
  const file = DriveApp.getFileById(tempFileId);
  const xlsxBlob = file.getBlob();
  xlsxBlob.setName("Azospirillum_Report.xlsx");

  //13. Send email with HTML body and Excel attachment
  MailApp.sendEmail({
    to: recipients.join(','),
    subject: "Azospirillum Report",
    htmlBody: `<p>Here is the latest Azospirillum report:</p>${htmlTable}`,
    attachments: [xlsxBlob]
  });

  //14. Clean up temp file
  DriveApp.getFileById(tempFileId).setTrashed(true);